In [1]:
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as py
import cPickle as pickle
np.random.seed = 0
%matplotlib inline
seed = 0

In [2]:
train = np.load('../../nn/data/train_scaled.npy')

#test_sub = np.load('../../nn/data/test_scaled.npy')

In [3]:
labels = train[:,0]   
train = train[:,1:]

In [4]:
zero_important_features = [3, 5, 15, 22, 23, 24, 25, 26, 35, 39, 45, 48, 49, 56, 57, 58, 59, 60,\
                           62, 68, 71, 72, 73, 74, 79, 80, 83, 88, 91, 94, 95, 96, 97, 118, 119, 120]
                           #6, 22, 27, 34, 46,
                           #5, 24, 31, 43, 50]

above_0_important_features = [ i for i in range(train.shape[1]) if i not in zero_important_features ]
train = train[:, above_0_important_features]
#test_sub = test_sub[:, above_0_important_features]

In [ ]:
selected_features_indices = [0, 1, 2, 4, 5, 6, 7, 8, 10, 12, 16, 18, 19, 20, 21, 27, 28, 29,\
                             31, 32, 33, 38, 40, 41, 42, 43, 51, 52, 55, 69, 75, 84, 85, 87,\
                             89, 90, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,\
                             110, 111, 112, 113, 114, 115, 116]
train = train[:,selected_features_indices]

In [ ]:
test_sub = test_sub[:,selected_features_indices]

In [4]:
test_size = 100000


#test_idx = np.random.randint(0, train.shape[0], test_size)
dev_data = train[:test_size,]
dev_labels = labels[:test_size,]

train_data = train[test_size:,]
train_labels = labels[test_size:,]



In [5]:
del train, labels




In [6]:
train_data.shape


(495212, 123)

In [7]:
# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

def gini(y_true, y_prob):
    y_true = np.asarray(y_true)    
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        
        y_i = y_true[i]
        
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    
    return gini


# Funcitons from olivier's kernel
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = - gini(labels, preds)
    return [('gini', gini_score)]




In [8]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False

EARLY_STOPPING_ROUNDS = 50  

params = {
    'max_delta_step' : 0.4,
    'scale_pos_weight' : 1.6, ##
    
    'min_child_weight' : 1, ##6
    'max_depth' : 2, ##4
    
    'gamma' : 9.8,##10
    
    
    'subsample' : .6,##.8
    'colsample_bytree' : .9,##.8
    'colsample_bylevel' : 1,
    
    'reg_alpha' : 5,##8, 4
    'reg_lambda' : 1,##1.3, 0.0006
    
    'learning_rate' : 0.1, ##
    'n_estimators' : 400,
    'objective' : "binary:logistic",
    'n_jobs' : -1,
    'eval_metric' : "auc"
    #'random_seed' : seed
}


In [ ]:
model = XGBClassifier(**params)
test_pred = np.zeros(test_sub.shape[0])
feature_imp = []
K = 5

kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)

for i, (train_indices, valid_indices) in enumerate(kfold.split(train,labels.reshape(-1))) :
#for i in range(K):
    x_train, y_train = train[train_indices], labels[train_indices]
    x_valid, y_valid = train[valid_indices], labels[valid_indices]
    
    eval_set = [(x_valid, y_valid)]
    #model = XGBClassifier(**params)
    st = np.datetime64('now')
    print "\nFold %d of %d " % (i+1, K) 
    
    _ = model.fit(x_train, y_train, eval_set=eval_set, eval_metric=gini_xgb, 
              early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose=False)
    #_ = model.fit(train_data,train_labels, verbose=False)

    print( "  Best N trees = ", model.best_ntree_limit )
    print( "  Best gini = ", - model.best_score )
    
    #pred = model.predict_proba(train_data)[:,1]
    #print "  Train score = ", gini(train_labels, pred)
    #pred = model.predict_proba(dev_data)[:,1]
    #print( "  Dev score = ", gini(dev_labels, pred) )
    print "It took : ", np.datetime64('now') - st
    
    #feature_imp.append(model.feature_importances_)
    
    test_pred += model.predict_proba(test_sub)[:,1]
    #if i == 4 :
    #    test_pred_5 = test_pred.copy()
    #    test_pred_5 /= 5
    #    print "  Writing submission after 5 iterations.."
    #    with open('../data/submission.csv', 'wb') as f :
    #        f.write('id,target')
    #        for x,p in zip(ids,test_pred_5) :
    #            f.write('\n%d,%.4f'%(x,p))
        
    
test_pred /= K

In [9]:
model = XGBClassifier(**params)
#test_pred = np.zeros(test_sub.shape[0])
feature_imp = []
K = 3

for i in range(K):
    
    model = XGBClassifier(**params)
    st = np.datetime64('now')
    print "\nFold %d of %d " % (i+1, K) 
    
    _ = model.fit(train_data,train_labels, verbose=False)

    
    pred = model.predict_proba(train_data)[:,1]
    print "  Train score = ", gini(train_labels, pred)
    pred = model.predict_proba(dev_data)[:,1]
    print( "  Dev score = ", gini(dev_labels, pred) )
    print "It took : ", np.datetime64('now') - st
    
    feature_imp.append(model.feature_importances_)
    
    #test_pred += model.predict_proba(test_sub)[:,1]
    #if i == 4 :
    #    test_pred_5 = test_pred.copy()
    #    test_pred_5 /= 5
    #    print "  Writing submission after 5 iterations.."
    #    with open('../data/submission.csv', 'wb') as f :
    #        f.write('id,target')
    #        for x,p in zip(ids,test_pred_5) :
    #            f.write('\n%d,%.4f'%(x,p))
        
    
#test_pred /= K


Fold 1 of 3 
  Train score =  0.298014910624
('  Dev score = ', 0.28171471832755657)
It took :  275 seconds

Fold 2 of 3 


KeyboardInterrupt: 

In [ ]:
ids = np.load('../../nn/data/test_ids.npy')

In [ ]:
with open('../data/submission1.csv', 'wb') as f :
	f.write('id,target')
    
	for i,p in zip(ids,test_pred) :
		f.write('\n%d,%.4f'%(i,p))

In [ ]:
with open('../data/model.pkl','wb') as f :
    pickle.dump(model, f)

-----------

# Choosing features by importance


In [ ]:
from sklearn.feature_selection import SelectFromModel
from beautifultable import BeautifulTable



In [ ]:
with open('../data/model.pkl', 'rb') as f:
    model = pickle.load(f)
    
    
    
    

In [ ]:
thresholds = np.sort([ f for f in set(i  for i in model.feature_importances_ if i > 0)])
zero_imp = sum([ 1  for i in model.feature_importances_ if i == 0])

print( "There are %d features with importance of 0, And they are removed.\nRemaining %d features."\
     % (zero_imp, train_data.shape[1] - zero_imp) )
                      
for thresh in thresholds :
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    selected_X = selection.transform(train_data)
      
    selection_model = XGBClassifier(**params)
    _ = selection_model.fit(selected_X, train_labels)
    
    selected_test = selection.transform(test_data)
    test_pred = selection_model.predict_proba(selected_test)[:,1]
    test_score = gini(test_labels, test_pred)
    
    print( "Threshold=%.5f, n=%d, Gini=%.3f" %(thresh, selected_X.shape[1], test_score) )
    
    


In [ ]:
#thresholds_to_test = thresholds[ (thresholds >= 0.00528) & (thresholds <= 0.00998)  ]


#thresholds_to_test = [ 0.00528169,  0.00586854,  0.0064554 ,  0.00704225,  0.00762911,

#        0.00880282,  0.00938967,  0.00997653 ]

thresholds_to_test = [0.000586, 0.00293, 0.00411, 0.00528 ]


In [ ]:
K = 5
SModel = XGBClassifier(**params)


for thresh in thresholds_to_test :
    
    
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    selected_X = selection.transform(train_data)
    selected_test = selection.transform(test_data)
    test_pred = np.zeros(test_data.shape[0])

    
    kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    
    print( "\n---------------\nThreshold=%.5f, n=%d ." %(thresh, selected_X.shape[1]) )
    Ntrees, best_score, valid_score = [], [], []
    st = np.datetime64('now')
    
    for i, (train_indices, valid_indices) in enumerate(kfold.split(train_data,train_labels.reshape(-1))) :
    
        x_train, y_train = selected_X[train_indices], train_labels[train_indices]
        x_valid, y_valid = selected_X[valid_indices], train_labels[valid_indices]
    
        eval_set = [(x_valid, y_valid)]
    
        print "."
        
    
        _ = SModel.fit(x_train, y_train, eval_set=eval_set, eval_metric=gini_xgb, 
                       
              early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose=False)

        Ntrees.append(SModel.best_ntree_limit)
        best_score.append(SModel.best_score)
        pred = SModel.predict_proba(x_valid)[:,1]
        valid_score.append(gini(y_valid, pred))
        
        test_pred += SModel.predict_proba(selected_test)[:,1]
    
    table = BeautifulTable()
    table.append_column('fold', range(K))
    table.append_column('best N trees', Ntrees)
    
    table.append_column('best score', best_score)
    
    table.append_column('valid score', valid_score)
    print(table)
    test_pred /= K
    print "  It took : ", np.datetime64('now') - st
    print "Gini score for test data is %.3f" % gini(test_labels, test_pred)
    
    
    


In [ ]:
# choosen threshold is 0.00411, n=54 .
#f_imp = [ [i,j] for i,j in enumerate(model.feature_importances_) ] 
#f_imp = sorted(f_imp,key= lambda d:d[1], reverse=True)[:55]


#selected_features_indices = [ i[0] for i in f_imp ]
selected_features_indices = [0, 1, 2, 4, 5, 6, 7, 8, 10, 12, 16, 18, 19, 20, 21, 27, 28, 29,\
                             31, 32, 33, 38, 40, 41, 42, 43, 51, 52, 55, 69, 75, 84, 85, 87,\
                             89, 90, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,\
                             110, 111, 112, 113, 114, 115, 116]

In [ ]:
feature_imp[0][0]


In [ ]:
feature_imp_set = []
for feature in feature_imp :
    
    feature_imp_set.append([ (i,j) for i,j in enumerate(feature) ])

In [ ]:
zero_imp = [ [], [], [] ]
for i,feature in enumerate(feature_imp) :
    for j,imp in enumerate(feature) :
        if imp == 0 :
            zero_imp[i].append(j)

In [ ]:
zero_imp

------

# Paramters Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
MAX_ROUNDS = 400
def custom_gini(y_true, y_pred):
    return gini(y_true, y_pred[:,1])
gini_scorer = make_scorer(custom_gini, greater_is_better = True, needs_proba=True)


params = {
    'max_delta_step' : 0.4,
    'scale_pos_weight' : 1.4, ##
    
    'max_depth' : 2, ##4
    'min_child_weight' : 1, ##6
    
    'gamma' : 9.8,##10
    
    
    'subsample' : .6,##.8
    'colsample_bytree' : .9,##.8
    'colsample_bylevel' : 1,
    
    'reg_alpha' : 5,##8, 4
    'reg_lambda' : 1,##1.3, 0.0006
    
    'learning_rate' : 0.1, ##
    'n_estimators' : 400,
    'objective' : "binary:logistic",
    'n_jobs' : -1,
    'eval_metric' : "auc",
    'random_seed' : seed
}

In [ ]:
# paramters combinations
cv_params1 = {
    'n_estimators' : [300,400,500],
    'learning_rate' : [0.1, 0.2, 0.3]
    
}
cv_params2 = {
    #'max_delta_step': [0, 0.3, 0.4, 0.5]
    #'scale_pos_weight': [1,1.2,1.4,1.6,1.8,2]
    'scale_pos_weight' : [1.3,1.4,1.5]
}
cv_params3 = {
    'max_depth' : range(2,11,2),
    'min_child_weight': range(1,8,2)
}
cv_params4 = {
    'max_depth' : [2,3,4],
    'min_child_weight': [1,2,3]
}
cv_params5 = {
    'gamma': [9.8,9.1,10]
    
}
cv_params6 = {
    'subsample': [0.56, 0.58, 0.6],
    'colsample_bytree': [ 0.7, 0.8, 0.9 ]
}
cv_params7 = {
 'subsample':[.35,.4,.45],
 'colsample_bylevel':[.35,.4,.45]
}
cv_params8 = {
 'subsample':[.35,.4,.45],
 'colsample_bylevel':[.35,.4,.45]
}
cv_params9 = {
 #'reg_alpha':[5,6,7],
 #'reg_lambda':[.0007, 0.008, 0.001, 0.1, 1]
    'reg_lambda' : [1, 1.5, 2]
    
}



In [ ]:

model = XGBClassifier(**params)

gsearch = GridSearchCV(model, param_grid = cv_params3,
                       
    cv = 3, verbose = 50, scoring=gini_scorer)


_ = gsearch.fit(train_data[:100000], train_labels[:100000])

print "best parameters : ", gsearch.best_params_, "\nbest score : ", gsearch.best_score_

pred = gsearch.predict_proba(dev_data)

print "dev score : ", gini(dev_labels, pred[:,1])
print "train scores : ", gsearch.cv_results_['mean_train_score']
print "test  scores : ", gsearch.cv_results_['mean_test_score']



In [ ]:

#model = XGBClassifier(**params)
#model.fit(train_data[:100000], train_labels[:100000])
pred = model.predict_proba(dev_data)

print gini(dev_labels, pred[:,1])


In [ ]:
pred = gsearch.predict_proba(test_data)

print gini(test_labels, pred[:,1])


In [ ]:
print "train scores : ", gsearch.cv_results_['mean_train_score']
print "test  scores : ", gsearch.cv_results_['mean_test_score']